Pobierz dane dotyczące scoringu kredytowego

*Pracujesz jako analityk danych w globalnej firmie finansowej. Przez lata firma gromadziła podstawowe dane bankowe i wiele informacji związanych z kredytami. Kierownictwo chce zbudować inteligentny system do segregowania ludzi w przedziałach oceny kredytowej, aby zmniejszyć wysiłek ręczny.*

**Zadanie na dziś: przygotować dane do modelu**

In [71]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

import danych

In [72]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train.csv"

# Load the latest version
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,
)

print("First 5 records:", df.head())

<ipython-input-72-db52f5663265>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,


First 5 records:        ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utiliz

/usr/local/lib/python3.11/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


**Poznaj dane:**

1.   sprawdź wymiary i typy kolumn
2.   usuń duplikaty
3.   usuń zdbędne kolumny
4.   sprawdz liczbę braków danych w poszczególnych kolumnach






In [73]:

print("Wymiary:", df.shape)
print("\nTypy kolumn:\n", df.dtypes)


df.drop_duplicates(inplace=True)
print("\nWymiary po usunięciu duplikatów:", df.shape)

if 'ID' in df.columns:
  df.drop('ID', axis=1, inplace=True)
  print("\nWymiary po usunięciu kolumny 'ID':", df.shape)
else:
  print("\nKolumna 'ID' nie istnieje w DataFrame.")

print("\nLiczba braków danych w poszczególnych kolumnach:\n", df.isnull().sum())


Wymiary: (100000, 28)

Typy kolumn:
 ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance            

Usuń kolumny, które nie będą wykorzystywane w analizie ['ID','Customer_ID','SSN','Name']

In [74]:
columns_to_drop = ['Customer_ID', 'SSN', 'Name']
for col in columns_to_drop:
    if col in df.columns:
        df.drop(col, axis=1, inplace=True)
        print(f"\nKolumna '{col}' została usunięta.")
        print("\nWymiary po usunięciu kolumny:", df.shape)
    else:
        print(f"\nKolumna '{col}' nie istnieje w DataFrame.")



Kolumna 'Customer_ID' została usunięta.

Wymiary po usunięciu kolumny: (100000, 26)

Kolumna 'SSN' została usunięta.

Wymiary po usunięciu kolumny: (100000, 25)

Kolumna 'Name' została usunięta.

Wymiary po usunięciu kolumny: (100000, 24)


usuń niektore z zbędnych znakow ('_','-'), zmień puste komorki na nan

In [75]:
# prompt: usuń niektore z zbędnych znakow ('_','-'), zmień puste komorki na nan

import numpy as np
# usuń niektore z zbędnych znakow ('_','-'), zmień puste komorki na nan
df.columns = df.columns.str.replace('_', ' ')
df.columns = df.columns.str.replace('-', ' ')
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)


Zmień nazwy miesiecy na zmienne liczbowe zgodnie z kolejnością miesięcy

In [77]:
month_mapping = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12
}

if 'Month' in df.columns:
    df['Month'] = df['Month'].map(month_mapping)
    print("\nZamiana nazw miesięcy na wartości liczbowe została zakończona.")
else:
    print("\nKolumna 'Month' nie istnieje w DataFrame.")



Zamiana nazw miesięcy na wartości liczbowe została zakończona.


Zakoduj zmienne tekstowe: 'Occupation', 'Type_of_Loan', 'Payment_Behaviour', 'Credit_Mix', użyj LabelEncodera

In [78]:
label_encoder = LabelEncoder()
for column in ['Occupation', 'Type of Loan', 'Payment Behaviour', 'Credit Mix']:
    if column in df.columns:
        df[column] = label_encoder.fit_transform(df[column].astype(str))
        print(f"Zakodowano kolumnę '{column}'.")
    else:
        print(f"Kolumna '{column}' nie istnieje w DataFrame.")


Zakodowano kolumnę 'Occupation'.
Zakodowano kolumnę 'Type of Loan'.
Zakodowano kolumnę 'Payment Behaviour'.
Zakodowano kolumnę 'Credit Mix'.


Przelicz kolumne Credit_History_Age na liczbę mięsiecy


In [80]:
# prompt: Przelicz kolumne Credit_History_Age na liczbę mięsiecy najprościej

import pandas as pd
import numpy as np
def convert_credit_history_age(age_str):
    if pd.isna(age_str):
        return np.nan
    years, months = map(int, age_str.split(' ')[0].split('Y'))
    return years * 12 + months

if 'Credit History Age' in df.columns:
    df['Credit History Age Months'] = df['Credit History Age'].apply(convert_credit_history_age)
    print(f"Dodano kolumnę 'Credit History Age Months'.")
else:
    print(f"Kolumna 'Credit History Age' nie istnieje w DataFrame.")


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# Rozdzielanie wartości po przecinku i usuwanie zbędnych spacji
df['Type_of_Loan'] = df['Type_of_Loan'].str.split(', ')

# Użycie MultiLabelBinarizer do One-Hot Encoding
mlb = MultiLabelBinarizer()
encoded_df = pd.DataFrame(mlb.fit_transform(df['Type_of_Loan']), columns=mlb.classes_)

# Łączenie z oryginalnym DataFrame
df = df.join(encoded_df).drop(columns=['Type_of_Loan'])

print(df.head())

Zmien inne zmienne na numeryczne

Uzupełnij braki w kolumnie Monthly_Inhand_Salary,  Annual_Income, .. średnią

Narysuj rozkłady zmiennych 'Annual_Income', 'Monthly_Balance'

usuń wartości odstające (w przypadku wieku wszystkie ujemne i te powyzej 100 lat)

Zwizualizuj korelacje między zmiennymi